In [35]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [36]:
from langchain.agents import tool
from typing import List, Dict, Annotated

In [37]:
from langchain_teddynote.tools import GoogleNews
from langchain_experimental.utilities import PythonREPL

In [38]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [39]:
from bs4 import BeautifulSoup
import requests
import re

### 도구 정의

In [40]:
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""

    print(f'검색어: {query}')

    news_tool = GoogleNews()

    return news_tool.search_by_keyword(query, k=2)

In [41]:
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    result = ""
    try:
        result = PythonREPL().run(code)
    except BaseException as e:
        print(f"Failed to execute. Error: {repr(e)}")
    finally:
        return result

### Agent 프롬프트 작성

In [42]:
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (                                           
            "system",
            "You are a helpful assistant. "             
            "Make sure to use the `search_news` tool for searching keyword related news."
            "Be sure to use the 'python_repl_tool' tool when make python code."
            "Be sure to use the 'naver_news_crawl' tool when naver news crawlling."
        ),
        ("placeholder", "{chat_history}"),          # 이전 대화 내용을 넣을 곳을 잡아둔다.
        ("human", "{input}"),                       # 사용자 입력

        # 에이전트가 검색하는 과정들이나 내용 등을 끄적이는 메모장 같은 공간을 플레이스 홀더로 만들어준다
        ("placeholder", "{agent_scratchpad}"),       
    ]
)

In [43]:
tools = [search_keyword, python_repl_tool]

### Agent 생성

In [44]:
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

agent = create_tool_calling_agent(llm, tools, prompt)

### AgentExecutor

In [45]:
agent_executor = AgentExecutor(
    agent=agent,                    # 각 단계에서 계획을 생성하고 행동을 결정하는 agent
    tools=tools,                    # agent 가 사용할 수 있는 도구 목록
    verbose=False,
    max_iterations=10,              # 최대 10번 까지 반복
    max_execution_time=10,          # 실행되는데 소요되는 최대 시간
    handle_parsing_errors=True      
)

In [46]:
def tool_callback(tool) -> None:
    print(f'=='* 20)
    print(f'[도구 호출]')
    print(tool)
    print(f"사용된 도구 이름 : {tool[-1].tool}") 
    print(f"입력 쿼리 : {tool[-1].tool_input}")
    print(f"Log: {tool[-1].log}")
    print(f'=='* 20)

In [47]:
def obervation_callback(observation) -> None:
    print(f'=='* 20)
    print(f'[관찰 내용')
    print(f"Observation: {observation[-1].observation}")
    print(f'=='* 20)

In [48]:
def result_callback(result: str) -> None:
    print(f'=='* 20)
    print(f'[최종 답변]')
    print(f"{result}")
    print(f'=='* 20)

In [49]:
result = agent_executor.stream({'input': '대구 동성로 떡볶이에 대해서 검색해주세요.'})

for step in result:
    if 'actions' in step:
        tool_callback(step['actions'])
    elif 'steps' in step:
        obervation_callback(step['steps'])
    elif 'output' in step:
        result_callback(step['messages'][-1].content)    
        

[도구 호출]
[ToolAgentAction(tool='search_keyword', tool_input={'query': '대구 동성로 떡볶이'}, log="\nInvoking: `search_keyword` with `{'query': '대구 동성로 떡볶이'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_uLNGdB4NQTbifIIPij4MrR80', 'function': {'arguments': '{"query":"대구 동성로 떡볶이"}', 'name': 'search_keyword'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac'}, id='run-0e900a14-ea08-4158-b499-17ff4e2f07e0', tool_calls=[{'name': 'search_keyword', 'args': {'query': '대구 동성로 떡볶이'}, 'id': 'call_uLNGdB4NQTbifIIPij4MrR80', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'search_keyword', 'args': '{"query":"대구 동성로 떡볶이"}', 'id': 'call_uLNGdB4NQTbifIIPij4MrR80', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_uLNGdB4NQTbifIIPij4MrR80')]
사용된 도구 이름 : search_keyword
입력 쿼리 : {'query': '대구 동성로 떡볶이'}
Log: 
Invoking: `search_keyword` with

In [50]:
result = agent_executor.stream({'input': '10 + 20을 출력하는 파이썬 코드를 작성하고 실행해주세요'})

for step in result:
    if 'actions' in step:
        tool_callback(step['actions'])
    elif 'steps' in step:
        obervation_callback(step['steps'])
    elif 'output' in step:
        result_callback(step['messages'][-1].content)

Python REPL can execute arbitrary code. Use with caution.


[도구 호출]
[ToolAgentAction(tool='python_repl_tool', tool_input={'code': 'print(10 + 20)'}, log="\nInvoking: `python_repl_tool` with `{'code': 'print(10 + 20)'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_57erkopMbBcw1imCq1BMqGgH', 'function': {'arguments': '{"code":"print(10 + 20)"}', 'name': 'python_repl_tool'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac'}, id='run-336604b9-7081-45c4-88e8-e6e59821aea1', tool_calls=[{'name': 'python_repl_tool', 'args': {'code': 'print(10 + 20)'}, 'id': 'call_57erkopMbBcw1imCq1BMqGgH', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'python_repl_tool', 'args': '{"code":"print(10 + 20)"}', 'id': 'call_57erkopMbBcw1imCq1BMqGgH', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_57erkopMbBcw1imCq1BMqGgH')]
사용된 도구 이름 : python_repl_tool
입력 쿼리 : {'code': 'print(10 + 20)'}
Log: 
I